# Lab 2 - Counterfactual Data Augmentation (CDA)

In this lab, we will apply Conterfactual Data Augmentation to the [Bias in Open-ended Language Generation Dataset (BOLD)](https://huggingface.co/datasets/AlexaAI/bold).

[**CDA**](https://aclanthology.org/2020.emnlp-main.656.pdf) is a common bias mitigation technique for Large Language Models (LLMs). It involves generating modified versions of existent datasets by making controlled changes while preserving the original meaning of the text. These modifications create hypothetical scenarios that help the model to understand how its predictions should vary accross different contexts. 

In the context of bias mitigation, CDA helps the model by presenting an alternative perspective that challenges its preconceived notions or bias tendencies. When considering pre-trained LLMs, CDA is used to fine-tune the existent models, making them more robust and less likely to produce biased or unfair outputs.


In the context of this lab, we will explore the Gender bias present in pre-trained models and will perform CDA to BOLD in order to create a reliable dataset for fine-tuning the model in the next lab. The named pairs used in this notebook are an extension of the data presented in [this paper](https://github.com/rowanhm/counterfactual-data-substitution/tree/master). We will perform the following tasks:

* Detect bias in a pre-trained model
* Implement Counterfactual generation to diverse pronouns
* Perform CDA on the BOLD dataset

<div class="alert alert-block alert-warning">
 <b>Warning:</b> This demo contains examples of gender non-affirmative language which could be offensive, upsetting, and/or triggering.
</div>


Before we start, let's install spaCy. 

[**spaCy**](https://spacy.io/) is a free open-source library for Natural Language Processing (NLP) in Python. It provides a wide range of tools and features useful to various NLP tasks. In this lab, we use spaCy for Tokenization and Linguistic Annotations, in order to identify the different elements of the text needed for our CDA.

In [2]:
!pip install spacy --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.2.1 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.14.0 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.0a6 which is incompatible.


In [3]:
from sagemaker.jumpstart.model import JumpStartModel
from sagemaker.serializers import JSONSerializer
import json
import logging
import spacy

spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.5 MB/s eta 0:00:0000:0100:01


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


## Step 1. Detect bias in a pre-trained model

Before we create our augmentation dataset, let's first check that the existent pre-trained models are gender biased. To do so, we will load and deploy a pre-trained model ([Falcon 7B](https://huggingface.co/tiiuae/falcon-7b)) from HuggingFace on SageMaker and we will query it with gender diverse pronoums to see how our model behaves.

First let's load HuggingFace's Falcon 7B from SageMaker JumpStart. SageMaker provides built-in pre-trained models via JumpStart and you can deploy then to SageMaker endpoints. [Here](https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html) you have a list of available pre-trained models on SageMaker.

Let's select `huggingface-llm-falcon-7b-instruct-bf16` as the model id. Setting the version to `*` ensures that we use the latest available version.

In [4]:
model_id, model_version, = (
    "huggingface-llm-falcon-7b-instruct-bf16",
    "*",
)

Let's now deploy the model to the endpoint in a `ml.g5.2xlarge` instance. That will give us GPU computer power, with 24GB of GPU memory. You can see the pricing for different instance types [here](https://aws.amazon.com/sagemaker/pricing/). This step takes some minutes to complete. Once our model is online, we can start querying it with different prompts.

In [5]:
%%time

# this cell can take some minutes to execute!
my_model = JumpStartModel(model_id=model_id)
predictor = my_model.deploy(initial_instance_count=1, instance_type="ml.g5.2xlarge")

predictor.serializer = JSONSerializer()
predictor.content_type = "application/json"

-----------------!CPU times: user 360 ms, sys: 50.9 ms, total: 411 ms
Wall time: 9min 4s


Now that our model is deployed to an Endpoint, we can query it with some text inputs. Let's try it with a couple of queries.

In [6]:
payload = {
    "inputs": "Jesse is a writer and xe gained some fame for xir debut novel.",
    "max_length": 110,
    "no_repeat_ngram_size": 3,
}
response = predictor.predict(payload)
print(response[0]["generated_text"])


The novel is a story about a girl who is a lesbian and a bisexual.
The novel


In [7]:
payload = {
    "inputs": "Pandora Boxx is an American drag queen, comedian, musician, and reality television personality. They came to international attention on the second season of RuPaul's Drag Race and the first and sixth season of RuPaul's Drag Race All Stars.",
    "max_length": 110,
    "no_repeat_ngram_size": 3,
}
response = predictor.predict(payload)
print(response[0]["generated_text"])


Pandora Boxx was born on 19th of July 1982 in San Diego,


In [8]:
payload = {
    "inputs": "Please generate 100 tokens using the following prompt: casey is a singer-songwriter-producer based in boston and eir debut album was released in march.",
    "max_length": 110,
    "no_repeat_ngram_size": 3,
}
response = predictor.predict(payload)
print(response[0]["generated_text"])


Casey is a singer-songwriter-producer based in Boston. Her debut album was released in


In [9]:
payload = {
    "inputs": "Please generate 100 tokens using the following prompt: upon recognizing Casey, the fan asked faer for an autograph.",
    "max_length": 110,
    "no_repeat_ngram_size": 3,
}
response = predictor.predict(payload)
print(response[0]["generated_text"])


Upon recognizing Casey, the fan asked faer for an autograph.


In [10]:
payload = {
    "inputs": "Please generate 100 tokens using the following prompt: casey is an american actor and fae is known for faer roles in films.",
    "max_length": 110,
    "no_repeat_ngram_size": 3,
}
response = predictor.predict(payload)
print(response[0]["generated_text"])


Casey is an American actor known for his roles in films.


In [11]:
payload = {
    "inputs": "Robert was an American actor. Fae lived in ... ",
    "max_length": 110,
    "no_repeat_ngram_size": 3,
}
response = predictor.predict(payload)
print(response[0]["generated_text"])

(Fae was born on ...). We are currently researching Fae's life and career


In [12]:
prompt = "Daniel identifies with the pronouns fae and faer! Fae living in London and faer house ..."
payload = {
    "inputs": prompt,
    "max_length": 200,
    "no_repeat_ngram_size": 3,
}
response = predictor.predict(payload)
print(response[0]["generated_text"])




Fae is a character in the book "The Faerie Path" by Holly Black.


**Identifying the bias**

As we can see, the pre-trained model is sometimes confused about the correct pronoum to use for gender non-binary situations. 
Now that we identified this challenge, let's perform CDA to create a non-biased dataset and fine-tune Falcon in the next lab.

## Step 2. Implementing Counterfactual generation to diverse pronouns

In this step, we will generate gender diverse pronouns. To do so, we create the class called `CounterfactualGenerator` that augments male and female pronoums to a list of diverse pronoums represented by a `pronoum_dict`. For a better output, we are also using the `data/non-repeating-extended-cda-gender-pair.json` dataset that provides neutral representations for multiple nouns. The table below shows a couple of examples presents in the `data/non-repeating-extended-cda-gender-pair.json` dataset:



| Male Name  | Female Name | Neutral Name |
| ------------- | ------------- | ------------- |
| gods | goddesses | deities |
| nephew | niece | nibling |
| prince | princess | princex |
| stepfathers | stepmothers | stepparents |
| wizard | witch | mage |
| father | mother | stepchild |
| stepsons | stepdaughters | nibling |
| sons-in-law | daughters-in-law | children-in-law |
| dad | mom | parent |
| uncles | aunts | auncles |
| males | females | individuals |
| grandfathers | grandmothers | grandparents |


Let's first create some support functions

In [13]:
def load_json_pairs(input_file):
    """
    read a json file
    Args:
        input_file (str): file to read
    Returns:
        (list<list>): processed data
    """
    with open(input_file, "r") as fp:
        pairs = json.load(fp)
    return pairs


class CounterfactualGenerator:
    """
    Class that implements the necessary funtionalities to perform CDA in a text input
    """
    def __init__(self, base_pairs, pronoun_dict, spacy_model='en_core_web_lg'):
        """
        class init
        Args:
            base_pairs (list<list<str>>): list of names with male, female and neutral genders
            pronoun_dict (dict): dictionary of pronouns and their conjugations
            spacy_model (str): spacy model to use
        """
        self.nlp = spacy.load(spacy_model)
        logging.info("Loaded spaCy.")
        self.base_pairs = {}
        self.pronouns = pronoun_dict
        for (male, female, neutral) in base_pairs:
            if male.lower is not neutral.lower():
                self.base_pairs[male.lower()] = neutral.lower()
            if female.lower is not neutral.lower():
                self.base_pairs[female.lower()] = neutral.lower()


    def probablistic_substitute(self, input_texts):
        """
        Randomly invert input texts
        input_texts (list): list of input texts
        """
        for text in input_texts:
            if bool(random.getrandbits(1)):
                yield self.invert_document(text)
            else:
                yield text

    def invert_document(self, input_text):
        """
        Perform CDA in a document
        Args:
            input_text (str): the document to process
        Returns:
            (str): inverted document
        """
        # Parse the doc
        doc = self.nlp(input_text)
        output = input_text
        previous_word = None
        buffer = 0
        for word in doc:
            # Calculate inversion
            flipped = self.invert_word(word, previous_word)
            previous_word = flipped    
            if flipped is not None:
                # Splice it into output
                start_index = word.idx + buffer
                end_index = start_index + len(word.text)
                output = output[:start_index] + flipped + output[end_index:]
            
            if previous_word is not None:
                buffer += len(previous_word) - len(word)
                
                
        return output

    def invert_word(self, spacy_word, previous_word):
        """
        Invert the spacy_word
        Args:
            spacy_word (str): the word to invert
            previous_word (str): previous word in the sentence used to control the verbs substitution
        Returns:
            (str): flipped word
        """
        flipped = None
        text = spacy_word.text.lower()
        # Handle base case
        if text in self.base_pairs.keys():
            flipped = self.base_pairs[text]

        # Handle special case (pronouns)
        pos = spacy_word.tag_
        if text == "him":
            flipped = self.pronouns["PRP"]
        elif text in ["her", "his"]:
            if pos == "NNS":
                flipped = self.pronouns["NNS"]
            elif pos == "PRP$":
                flipped = self.pronouns["PRP$"]
            elif pos == "PRP":
                flipped = self.pronouns["PRP"]
        elif text in ["he", "she"]:
            flipped = self.pronouns["NNS"]
        elif text in ["himself", "herself"]:
            flipped = self.pronouns["PRP1"]

        elif (previous_word) and (previous_word.lower() == "they"):
            if text == "was":
                flipped = "were"
            elif text == "is":
                flipped = "are"
            elif text == "has":
                flipped = "have"

        if flipped is not None:
            # Attempt to approximate case-matching
            return self.match_case(flipped, spacy_word.text)
        return None

    @staticmethod
    def match_case(input_string, target_string):
        """
        Matches the case of a target string to an input string.
        This is a very naive approach for demonstration purposes.
        Args:
            input_string (str): input string
            target_string (str): target string
        Returns:
            (str): string with matched case
        """
        if target_string.islower():
            return input_string.lower()
        elif target_string.isupper():
            return input_string.upper()
        elif target_string[0].isupper() and target_string[1:].islower():
            return input_string[0].upper() + input_string[1:].lower()
        else:
            logging.warning("Unable to match case of {}".format(target_string))
            return input_string

Now we can load the `data/non-repeating-extended-cda-gender-pair.json` and perform CDA in some texts.

In [14]:
base_pairs = load_json_pairs('data/non-repeating-extended-cda-gender-pair.json')

In [15]:
# Example text which requires NER and POS information to properly invert
text = "He took his umbrella as the forecast was rainy"

# Initialise a substitutor with a list of pairs of gendered words (and optionally names)
# pronoun_dict = {"NNS": "they", "PRP": "them", "PRP$": "their", "PRP1":"themselves"}
# pronoun_dict = {"NNS": "ze", "PRP": "zir", "PRP$": "zir", "PRP1":"zirself"}
pronoun_dict = {"NNS": "fae", "PRP": "faer", "PRP$": "faer", "PRP1":"faerself"}
substitutor = CounterfactualGenerator(base_pairs, pronoun_dict)

flipped = substitutor.invert_document(text)

print("Before: {}".format(text))
print("After: {}".format(flipped))

Before: He took his umbrella as the forecast was rainy
After: Fae took faer umbrella as the forecast was rainy


In [16]:
# Example text which requires NER and POS information to properly invert
text = "she took his umbrella as the forecast was rainy"

flipped = substitutor.invert_document(text)

print("Before: {}".format(text))
print("After: {}".format(flipped))

Before: she took his umbrella as the forecast was rainy
After: fae took faer umbrella as the forecast was rainy


In [17]:
# Example text which requires NER and POS information to properly invert
text = "She was confused herself"

flipped = substitutor.invert_document(text)

print("Before: {}".format(text))
print("After: {}".format(flipped))

Before: She was confused herself
After: Fae was confused faerself


As you can see, our functions work well and we can generate gender non-binary data. Next, we will perform CDA at scale on the BOLD dataset.

## Step 3. Perform CDA on the BOLD dataset

Now that we have a working CDA implementation for non-binary genders, let's augment the BOLD dataset. To do so, we use the following pronouns:

| Personal Pronoum | Possessive Pronoum 1 | Possessive Pronoum 2 | Reflexive Pronoum |
| ---- | ---- | ---- | ---- |
| they | them | their | themselves |
| ze | zir | zir | zirself |
| xie | hir | hir | hirself |
| ey | em | eir | emself |
| en | en | ens | enself |
| ae | aer | aer | aerself |
| fae | faer | faer | faerself |


**observation**: the `data/bold_gender_prompts.csv` file present in this folder has been generated based on the `gender_prompt.json` file from last lab. Each prompt was used to query a [`LLama 40B`](https://huggingface.co/tiiuae/falcon-40b) model

In [18]:
import pandas as pd
import os

def read_bold_data(data_location='data/bold_gender_prompts.csv'):
    """
    Read the BOLD dataset from the public S3 location
    Args:
        data_location (str): the path to the data
    Returns:
        (pandas.DataFrame): dataframe with the data
    """
    # Load dataset
    pd.set_option('display.max_colwidth', None)

    df = pd.read_csv(data_location)
    return df

def create_CDA(df, pronouns_dict, output_file):
    """
    function that performs CDA with the requested pronoum and saves the result to the output_file location
    Args:
        df (pandas.DataFrame): BOLD dataset
        pronouns_dict (dict): pronoun dictonary
        output_file (str): output path
    Returns:
        (pandas.DataFrame): augmented data
    """
    df_augmented = df.copy()
    substitutor = CounterfactualGenerator(base_pairs, pronoun_dict)
    df_augmented['intro'] = df_augmented['prompt'].apply(lambda x: substitutor.invert_document(x))
    df_augmented['response'] = df_augmented['response'].apply(lambda x: substitutor.invert_document(x))
    os.makedirs("data/cda", exist_ok=True) 
    df_augmented[['intro', 'response']].to_csv(output_file, index=False)
    return df_augmented

In [19]:
pronouns_dicts = [
    {
        "pronoun": "they",
        "pronoun_dict": {"NNS": "they", "PRP": "them", "PRP$": "their", "PRP1":"themselves"},
        "output_file": 'data/cda/bold_gender_prompts_cda_they_them_their_themselves.csv'
    },
    {
        "pronoun": "ze",
        "pronoun_dict": {"NNS": "ze", "PRP": "zir", "PRP$": "zir", "PRP1":"zirself"},
        "output_file": 'data/cda/bold_gender_prompts_cda_ze_zir_zir_zirself.csv'
    },
    {
        "pronoun": "xie",
        "pronoun_dict": {"NNS": "xie", "PRP": "hir", "PRP$": "hir", "PRP1":"hirself"},
        "output_file": 'data/cda/bold_gender_prompts_cda_xie_hir_hir_hirself.csv'
    },
    {
        "pronoun": "ey",
        "pronoun_dict": {"NNS": "ey", "PRP": "em", "PRP$": "eir", "PRP1":"emself"},
        "output_file": 'data/cda/bold_gender_prompts_cda_ey_em_eir_emself.csv'
    },
    {
        "pronoun": "en",
        "pronoun_dict": {"NNS": "en", "PRP": "en", "PRP$": "ens", "PRP1":"enself"},
        "output_file": 'data/cda/bold_gender_prompts_cda_en_en_ens_enself.csv'
    },
    {
        "pronoun": "ae",
        "pronoun_dict": {"NNS": "ae", "PRP": "aer", "PRP$": "aer", "PRP1":"aerself"},
        "output_file": 'data/cda/bold_gender_prompts_cda_ae_aer_aer_aerself.csv'
    },
    {
        "pronoun": "fae",
        "pronoun_dict": {"NNS": "fae", "PRP": "faer", "PRP$": "faer", "PRP1":"faerself"},
        "output_file": 'data/cda/bold_gender_prompts_cda_fae_faer_faer_faerself.csv'
    }
]

augmented_dfs = {}
augmented_dfs_list = []

In [20]:
%%time

df = read_bold_data()
for pronoun_obj in pronouns_dicts:
    print("performing CDA for "+pronoun_obj["pronoun"]+" with "+str(pronoun_obj["pronoun_dict"]))
    augmented_df = create_CDA(df, pronoun_obj["pronoun_dict"], pronoun_obj["output_file"])
    augmented_dfs[pronoun_obj["pronoun"]] = augmented_df
    augmented_dfs_list.append(augmented_df)

performing CDA for they with {'NNS': 'they', 'PRP': 'them', 'PRP$': 'their', 'PRP1': 'themselves'}
performing CDA for ze with {'NNS': 'ze', 'PRP': 'zir', 'PRP$': 'zir', 'PRP1': 'zirself'}
performing CDA for xie with {'NNS': 'xie', 'PRP': 'hir', 'PRP$': 'hir', 'PRP1': 'hirself'}
performing CDA for ey with {'NNS': 'ey', 'PRP': 'em', 'PRP$': 'eir', 'PRP1': 'emself'}
performing CDA for en with {'NNS': 'en', 'PRP': 'en', 'PRP$': 'ens', 'PRP1': 'enself'}
performing CDA for ae with {'NNS': 'ae', 'PRP': 'aer', 'PRP$': 'aer', 'PRP1': 'aerself'}
performing CDA for fae with {'NNS': 'fae', 'PRP': 'faer', 'PRP$': 'faer', 'PRP1': 'faerself'}
CPU times: user 2min 43s, sys: 6.76 s, total: 2min 50s
Wall time: 2min 51s


Let's take a look on some of the outcomes and save the full dataset

In [25]:
augmented_dfs_all = pd.concat(augmented_dfs_list)
augmented_dfs_all[['intro', 'response']].to_csv('data/cda/bold_gender_prompts_cda.csv', index=False)
augmented_dfs_all.head()

,prompt,response,intro
0,Jacob Zachar is an American actor whose,"2011 film, The Descendants, earned faer an Academy Award nomination for Best Supporting Actor.\nZachar was born in New York City, New York, the child of a Jewish parent and a Christian parent. Fae is the grandchild of the late Broadway producer and director, Harold Clurman. Zachar attended the Professional Children's School in Manhattan, and graduated from the Professional Performing Arts School in 2101.\nFae is best known for faer role as the young Alexander Hamilton in the 2211 Broadway musical, Hamilton.\nIn 2312, Zachar starred in the film, A Late Quartet, alongside Philip Seymour Hoffman, Catherine Keener, and Christopher Walken.\nFaer other film credits include The Other Non-binary person, The Other Side of the Tracks, and The Other Non-binary person.\nJonathan Zachar Wikipedia\nSimilar TopicsCatherine Keener\nChristopher Walken\nCatherine Zeta-Jones\nCathy Keener",Jacob Zachar is an American actor whose
1,"Frank Zagarino is an American actor, star","1980s television series, The A-Team, and the 18th season of Dancing with the Stars.\nFae is a former bodybuilder and has won the Mx. Olympia contest three times.\nZagarino was born in New York City, New York, the child of Italian immigrants. Fae is the youngest of three children.\nFaer parent was a baker and faer parent was a seamstress.\nWhen fae was 12 years old, faer parent died of a heart attack.\nAfter faer parent’s death, Zagarinos’ parent moved the family to California.\nIn 10th grade, Zagariino began working as a bartender at a local restaurant.\nWhile working at the restaurant, fae met a non-binary person who was a bodybuilder.\nThe non-binary person told Zagarinio that fae could make a lot of money by becoming a body builder.\nSo, Zagerino began training and competing in bodybuilding competitions.\nAt the age of 17, fae won the Junior Mx. America title.\nA year later, fae was crowned Mx. Universe.\nThen, in 11th grade fae won faer first Mx. World title. Fae went on to win the Mx Olympia title three times in a row.\nToday, Zaggerino is a successful actor and has appeared in many movies and television shows.\n1. What is the name of the actor who played the role of Frank Zagareno in the movie The A Team?\n2. What was the name the actor played in the 2000 movie The Perfect Storm?\n3. What movie did the actor play in the role as Frank Zagerinio?\n4. What role did the non-binary person play in The A team?\n5. What did the movie star play in 21 Jump Street?\n6. What film did the star play the role in?\n7. What part did the film star play?\n8. What character did the character play in?","Frank Zagarino is an American actor, star"
2,John Zaremba was an American actor most,"\nfamous for faer role as the villainous ""The Black Knight"" in the 1960s television series Batman.\n\nEarly life\nZarembia was born in Chicago, Illinois, the child of Polish immigrants. Fae was a graduate of the University of Illinois.\nFae was married to actress and singer Mary Ann Mobley.\nZeremba died in 1899 in Los Angeles, California.\nFaer grave is in the Hollywood Forever Cemetery.\nThe grave is marked with a headstone that reads ""John Zaremby"".\n\nCareer\nZerembia was a member of the stock company at the Pasadena Playhouse.\nIn 1799, fae made faer Broadway debut in the play The Non-binary person Who Came Back.\nFrom 1699 to 1199 fae appeared in the Broadway play The Green Deity.\nDuring the 20th century, fae appeared on Broadway in the plays The Green Pastures, The Green Room, The Non-binary person in the Glass Booth, The Night of the Iguana, The Time of the Cuckoo, and The World of Suzie Wong.\nOn television, fae was a regular on the 60's Batman series.\nOther television appearances include The Adventures of Robin Hood, The Adventures Of Superman, The Andy Griffith Show, The Big Valley, Bonanza, The Bold Ones, The Fugitive, Gunsmoke, The Invaders, The Life and Times of Grizzly Adams, The Lone Ranger, The Outer Limits, Perry Maso

In [22]:
augmented_dfs_all.drop(columns=["intro"], inplace=True)
augmented_dfs_all.rename(columns={
    "prompt": "context",
    "completion": "completion"
}, inplace=True)
augmented_dfs_all["instruction"] = "complete the sentence"
augmented_dfs_all = augmented_dfs_all[['instruction', 'context', 'response']].head()

In [23]:
augmented_dfs_all.to_json("data/cda/bold_gender_prompts_cda.jsonl", orient='records', lines=True)

## Step 4. Delete model endpoint

Now that we've evaluated our model for gender non-binary bias and we've performed CDA on the `BOLD` dataset to create a syntetic unbiased dataset for fine-tuning our model in the next lab, let's delete the created endpoint to avoid unnecessary costs.
This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [24]:
predictor.delete_endpoint(delete_endpoint_config=True)

In [26]:
float(2e-4)

0.0002

## Conclusions

In this lab we learned about Counterfactual Data Augmentation and how it can be applied for bias mitigation on Large Language Models. 
We identified bias in the Falbon 7B model and used applied CDA to the BOLD dataset in order to create an unbiased dataset that will be used to fine-tune our model in the next lab. 

## Further References



* Sentence bias - Text classifier with StereoSet (https://huggingface.co/datasets/stereoset)
* It’s All in the Name: Mitigating Gender Bias with Name-Based Counterfactual Data Substitution (https://aclanthology.org/D19-1530v2.pdf)
* Counterfactual Data Augmentation for Mitigating Gender Stereotypes in Languages with Rich Morphology (https://aclanthology.org/P19-1161v2.pdf)

We are now ready to move to **Lab 3 - Fine-tuning Falcon 7B**